<a href="https://colab.research.google.com/github/kerupinen/Compdb_final_repo/blob/main/Final_Project_Rupinen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install pymongo

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 676.9/676.9 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.7/307.7 kB 19.3 MB/s eta 0:00:00


In [5]:
import pymongo
from pymongo import MongoClient

client = pymongo.MongoClient(host='3.230.203.12',  # host is the hostname for the database
                                port=27018,  # port is the port number that mongo is running on
                                username='compdb_student',  # username for the db
                                password='compdbpass',  # password for the db
                                authSource='admin')  # Since our user only exists for the emse6992 db, we need to specify this

# Setup collections and connections
db = client.twitter
stats_coll = db.twitter_statuses
retweets_coll = db.twitter_retweets
favs_coll = db.twitter_favorites
lists_coll = db.twitter_lists_updated
friends_coll = db.twitter_friends_updated


In [7]:
import psycopg2
import psycopg2.extras
import pandas as pd

# Connect to the database
connection = psycopg2.connect(host='3.230.203.12',
                             user='compdb',
                             port=5438,
                             database='twitter',
                             password='compdbs_postgres')
connection.set_session(readonly=True, autocommit=True)

cursor = connection.cursor(cursor_factory = psycopg2.extras.RealDictCursor)

In [27]:
query = 'SELECT extract(isodow FROM created_date) FROM twitter.statuses'
cursor.execute(query) #Executing query
res = cursor.fetchall()

res[1]



RealDictRow([('extract', Decimal('4'))])

In [ ]:
mylist=[]
for i in res:
    mylist.append(i['extract'])
print(mylist)


In [117]:
x = 1
weekdayCountList=[]
total = 0
ototal = 0
while x < 8:
    query = "SELECT count(distinct(statuses)) FROM twitter.statuses WHERE extract(isodow FROM created_date) = '%d'" % x
    cursor.execute(query) #Executing query
    res = cursor.fetchall()

    weekdayCountList.append([x,res[0]['count']])

    x=x+1


print(weekdayCountList)


[[1, 6618], [2, 6264], [3, 6684], [4, 6647], [5, 6729], [6, 5172], [7, 4717]]


In [114]:
import plotly.express as px
fig = px.bar(weekdayCountList, x=0, y=1, title = "Tweets by weekday",labels={'0':'Day of Week','1':'Number of Tweets'}, range_y=(4000,7000))
fig.show()

In [111]:
x = 1
weekDayTotals=[]
total = 0
ototal = 0
while x < 8:
    query = "SELECT distinct(status_id),favorites_count FROM twitter.statuses WHERE extract(isodow FROM created_date) = '%d'" % x
    cursor.execute(query) #Executing query
    res = cursor.fetchall()

    for i in res:
        total = total+i['favorites_count']

    query = "SELECT distinct(status_id),retweet_count FROM twitter.statuses WHERE extract(isodow FROM created_date) = '%d'" % x
    cursor.execute(query) #Executing query
    res = cursor.fetchall()

    for w in res:
        ototal = ototal+w['retweet_count']

    weekDayTotals.append([x,total,ototal])
    x=x+1


print(weekDayTotals)


[[1, 18805170, 8095532], [2, 39772650, 16901625], [3, 62421716, 27924833], [4, 83495056, 39152366], [5, 111754829, 50272127], [6, 132327833, 67034962], [7, 154817036, 75942274]]


In [126]:
x = 1
avgRetweetpd = []
avgLikespd = []
while x < 8:
    average = (weekDayTotals[x-1][2])/(weekdayCountList[x-1][1])
    avgRetweetpd.append([x,average])

    average = (weekDayTotals[x-1][1])/(weekdayCountList[x-1][1])
    avgLikespd.append([x,average])
    x=x+1

print(avgRetweetpd)
print(avgLikespd)


1223.25959504382
2841.5185856754306
2698.2159961685825
6349.401340996169
[[1, 1223.25959504382], [2, 2698.2159961685825], [3, 4177.86250748055], [4, 5890.2310816909885], [5, 7470.965522365879], [6, 12961.129543696828], [7, 16099.697689209243]]
[[1, 2841.5185856754306], [2, 6349.401340996169], [3, 9338.97606223818], [4, 12561.314277117497], [5, 16607.940109971765], [6, 25585.42788089714], [7, 32821.08034767861]]


In [135]:
fig = px.bar(avgRetweetpd, x=0, y=1, title = "retweet per tweet by weekday",labels={'0':'Day of Week','1':'Retweets per Tweet'}, range_y=(0,17000))
fig.show()

In [136]:
fig = px.bar(avgLikespd, x=0, y=1, title = "like per tweet by weekday",labels={'0':'Day of Week','1':'Like per Tweet'}, range_y=(0,33000))
fig.show()

In [83]:
#First Week
query = 'SELECT status_id FROM twitter.statuses WHERE extract(day FROM created_date) <= 7'
cursor.execute(query) #Executing query
res = cursor.fetchall()

#Second week
query = 'SELECT created_date FROM twitter.statuses WHERE extract(day FROM created_date) BETWEEN 8 and 14'
cursor.execute(query) #Executing query
res = cursor.fetchall()
#Third Week
query = 'SELECT status_id FROM twitter.statuses WHERE extract(day FROM created_date) BETWEEN 15 and 21'
cursor.execute(query) #Executing query
res = cursor.fetchall()

#Last Week
query = 'SELECT status_id FROM twitter.statuses WHERE extract(day FROM created_date) >21'
cursor.execute(query) #Executing query
res = cursor.fetchall()

[RealDictRow([('created_date', datetime.date(2020, 4, 12))]), RealDictRow([('created_date', datetime.date(2020, 3, 11))]), RealDictRow([('created_date', datetime.date(2020, 3, 11))]), RealDictRow([('created_date', datetime.date(2020, 1, 10))]), RealDictRow([('created_date', datetime.date(2019, 9, 13))]), RealDictRow([('created_date', datetime.date(2019, 9, 10))]), RealDictRow([('created_date', datetime.date(2019, 9, 8))]), RealDictRow([('created_date', datetime.date(2019, 7, 14))]), RealDictRow([('created_date', datetime.date(2019, 4, 11))]), RealDictRow([('created_date', datetime.date(2019, 4, 10))]), RealDictRow([('created_date', datetime.date(2021, 1, 11))]), RealDictRow([('created_date', datetime.date(2019, 3, 14))]), RealDictRow([('created_date', datetime.date(2019, 3, 11))]), RealDictRow([('created_date', datetime.date(2019, 3, 10))]), RealDictRow([('created_date', datetime.date(2019, 2, 13))]), RealDictRow([('created_date', datetime.date(2019, 2, 11))]), RealDictRow([('created_d